# Prepare SeisMonitor
Clone some mandatory repositories

In [1]:
import os

def prepare_seismonitor(seismo_folder):
    """
    Parameters:
    -----------
    seismo_folder: str
        Installation folder path
    """
    eqt = os.path.join(seismo_folder,"EQTransformer")
    pnet = os.path.join(seismo_folder,"PhaseNet")
    seismo = os.path.join(seismo_folder,"SeisMonitor")

    if not os.path.isdir(seismo_folder):
        os.makedirs(seismo_folder)
    if not os.path.isdir(pnet):
        os.makedirs(pnet)
        os.system(f"git clone https://github.com/ecastillot/PhaseNet.git {pnet}")
    if not os.path.isdir(eqt):
        os.makedirs(eqt)
        os.system(f"git clone https://github.com/smousavi05/EQTransformer.git {eqt}")
    if not os.path.isdir(seismo):
        os.makedirs(seismo)
        os.system(f"git clone https://github.com/ECP-Sismologia/SeisMonitor.git {seismo}")

seismopath = "/home/emmanuel/test/seismo"
prepare_seismonitor(seismopath)

# SeisMonitor
Import SeisMonitor in your sys packacges.

In [2]:
import sys
seismonitor = os.path.join(seismopath,"SeisMonitor")
sys.path.insert(0,seismonitor)

- SeisMonitor supports FDSN or SDS clients. 
- resp file is needed if your client don't have get_stations method
- out_folder to save all outputs

In [3]:
# route = "/home/emmanuel/archive/sds"
route = 'http://sismo.sgc.gov.co:8080'
resp = os.path.join("../data","metadata/CM.xml")
out_folder = "../out/scanloc/test_VMM"

# outs
info_dir = os.path.join(out_folder,'downloads')
picks_dir = os.path.join(out_folder,'detections')
events_dir = os.path.join(out_folder,'events')

## locator parameters (SEISAN software is mandatory to locate events)
vel_file = os.path.join("../data","vel_model/CM_vel1d.csv")
station0_file = os.path.join("../data","CM/STATION0.HYP")

In [4]:
from obspy.clients.fdsn import Client
from obspy.core.utcdatetime import UTCDateTime
from SeisMonitor.downloader.utils import DownloadRestrictions

client = Client(route)
restrictions = DownloadRestrictions(network="CM",
                          station="AGCC,EZNC,SNPBC,MORC,OCNC,SML1C,VMM*,BRR*,LL*",
                          location="*",
                          channel="*",
                          starttime=UTCDateTime("2017-10-15T06:00:00.0"),
                          endtime=UTCDateTime("2017-10-15T08:00:00.0"),
                          chunklength_in_sec=43200,
                          overlap_in_sec=None,
                          groupby='{network}.{station}.{location}',
                          threshold=60,
                          location_preferences=["","00","20","10"],
                          channel_preferences=["HH","BH"],
                          to_pick=(10,0.3))                    

## EQTransformer

### Run

In [12]:
from SeisMonitor.scanloc.utils import EQTobj
from SeisMonitor.scanloc.monitor import Monitor

eqt_model = os.path.join(seismopath,'EQTransformer/ModelsAndSampleData/EqT_model.h5')
eqtobj = EQTobj(model_path = eqt_model,
            chunk_size = 3600,
            n_processor = 4,
            overlap = 0.3,
            detection_threshold =0.1,
            P_threshold = 0.01,
            S_threshold = 0.01,
            batch_size = 10,
            number_of_plots = 1,
            plot_mode = 1 )  
            
monitor = Monitor(providers = [client],
                    restrictions=restrictions,
                    info_dir=info_dir,
                    picks_dir=picks_dir,
                    events_dir=events_dir)

monitor.make_json(from_xml=resp)
monitor.download()
monitor.eqt_picker(eqtobj,rm_downloads=False)
monitor.eqt_associator()

01-11 19:27 [INFO] [json] running to create json
01-11 19:27 [INFO] [json] ['CM-AGCC', 'CM-AGCC', 'CM-EZNC', 'CM-SNPBC', 'CM-OCNC', 'CM-SML1C', 'CM-VMM10', 'CM-VMM09', 'CM-VMM05', 'CM-VMM12', 'CM-VMM11', 'CM-VMM07', 'CM-BRR', 'CM-LL8C', 'CM-LL6C', 'CM-LL5C', 'CM-LL1C', 'CM-LL7C'] ok
01-11 19:27 [INFO] [json] Total time of execution: 3.472363 seconds
01-11 19:27 [INFO] [Downloader] Running to download
01-11 19:27 [INFO] [Downloader: chunktime] Total chunktime list: 1
01-11 19:27 [INFO] [Downloader: chunktime] 1 of 1 -> starttime=2017-10-15T06:00:00.000000Z | endtime=2017-10-15T08:00:00.000000Z
01-11 19:27 [INFO] [CM-VMM10: ] available:[('00', 'HH?')], selected [('00', 'HH?')] according to preference
01-11 19:27 [INFO] [CM-VMM12: ] available:[('00', 'HH?')], selected [('00', 'HH?')] according to preference
01-11 19:27 [INFO] [CM-AGCC: ] available:[('00', 'HH?')], selected [('00', 'HH?')] according to preference
01-11 19:27 [INFO] [CM-VMM11: ] available:[('00', 'HH?')], selected [('00', '

01-11 19:28 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 6.75 seconds.
01-11 19:28 [INFO] [EQTransformer] *** Detected: 5 events.
01-11 19:28 [INFO] [EQTransformer]  *** Wrote the results into --> " /home/emmanuel/test/seismo/SeisMonitor/examples/../out/scanloc/test/detections/eqt/AGCC_outputs "
01-11 19:28 [INFO] [EQTransformer] Started working on EZNC, 2 out of 13 ...
01-11 19:28 [INFO] [EQTransformer] 20171015T060000Z__20171015T080000Z.mseed


01-11 19:28 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 1.69 seconds.
01-11 19:28 [INFO] [EQTransformer] *** Detected: 3 events.
01-11 19:28 [INFO] [EQTransformer]  *** Wrote the results into --> " /home/emmanuel/test/seismo/SeisMonitor/examples/../out/scanloc/test/detections/eqt/EZNC_outputs "
01-11 19:28 [INFO] [EQTransformer] Started working on LL5C, 3 out of 13 ...
01-11 19:28 [INFO] [EQTransformer] 20171015T060000Z__20171015T080000Z.mseed


01-11 19:28 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 1.59 seconds.
01-11 19:28 [INFO] [EQTransformer] *** Detected: 2 events.
01-11 19:28 [INFO] [EQTransformer]  *** Wrote the results into --> " /home/emmanuel/test/seismo/SeisMonitor/examples/../out/scanloc/test/detections/eqt/LL5C_outputs "
01-11 19:28 [INFO] [EQTransformer] Started working on LL6C, 4 out of 13 ...
01-11 19:28 [INFO] [EQTransformer] 20171015T060000Z__20171015T080000Z.mseed


01-11 19:28 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 1.54 seconds.
01-11 19:28 [INFO] [EQTransformer] *** Detected: 0 events.
01-11 19:28 [INFO] [EQTransformer]  *** Wrote the results into --> " /home/emmanuel/test/seismo/SeisMonitor/examples/../out/scanloc/test/detections/eqt/LL6C_outputs "
01-11 19:28 [INFO] [EQTransformer] Started working on LL7C, 5 out of 13 ...
01-11 19:28 [INFO] [EQTransformer] 20171015T060000Z__20171015T080000Z.mseed


...

01-11 19:28 [INFO] [EQTransformer] Finished the prediction in: 0 hours and 0 minutes and 1.73 seconds.
01-11 19:28 [INFO] [EQTransformer] *** Detected: 4 events.
01-11 19:28 [INFO] [EQTransformer]  *** Wrote the results into --> " /home/emmanuel/test/seismo/SeisMonitor/examples/../out/scanloc/test/detections/eqt/VMM12_outputs "
01-11 19:28 [INFO] [EQTransformer] Total time of execution: 57.696362 seconds
  0%|          | 0/480 [00:00<?, ?it/s]

reading AGCC_outputs ...
reading EZNC_outputs ...
reading LL5C_outputs ...
reading LL6C_outputs ...
reading LL7C_outputs ...
reading LL8C_outputs ...
reading SNPBC_outputs ...
reading VMM05_outputs ...
reading VMM07_outputs ...
reading VMM09_outputs ...
reading VMM10_outputs ...
reading VMM11_outputs ...
reading VMM12_outputs ...


 86%|████████▋ | 415/480 [00:00<00:00, 1298.18it/s]

The Number of Associated Events: 5

5 Event(s) in Catalog:
2017-10-15T06:35:11.670000Z |  +0.000,   +0.000
2017-10-15T06:47:16.580000Z |  +0.000,   +0.000
2017-10-15T06:51:56.480000Z |  +0.000,   +0.000
2017-10-15T06:57:24.000000Z |  +0.000,   +0.000
2017-10-15T07:37:29.650000Z |  +0.000,   +0.000


100%|██████████| 480/480 [00:20<00:00, 1298.18it/s]

### outs

In [13]:
eqt_detections = os.path.join(out_folder,"detections","eqt")
os.listdir(eqt_detections)

['traceNmae_dic.json',
 'Y2000.phs',
 'associations.xml',
 'LL6C_outputs',
 'VMM11_outputs',
 'LL7C_outputs',
 'SNPBC_outputs',
 'EZNC_outputs',
 'VMM05_outputs',
 'VMM09_outputs',
 'AGCC_outputs',
 'LL5C_outputs',
 'LL8C_outputs',
 'VMM12_outputs',
 'VMM10_outputs',
 'VMM07_outputs']

## PhaseNet

### run
PhaseNet take more time because there are 3000 samples + all overlaped windows. Furthermore, It cosumes much more RAM.

PhaseNet parallelize by stations and not by all overlaped windows in one station. Then, It consumes more time and more RAM.

*PhaseNet is really useful for local networks*

In [14]:
from SeisMonitor.scanloc.utils import PhaseNetobj
from SeisMonitor.scanloc.monitor import Monitor
pnet_model = os.path.join(seismopath,'PhaseNet/model/190703-214543')
pnetobj = PhaseNetobj(model_path = pnet_model ,
                mode='pred',
                P_threshold=0.1,
                S_threshold=0.1,
                batch_size=1, 
                plot=False, 
                save_result=False) 
monitor = Monitor(providers = [client],
                    restrictions=restrictions,
                    info_dir=info_dir,
                    picks_dir=picks_dir,
                    events_dir=events_dir)
monitor.make_json(from_xml=resp)
monitor.download()
monitor.mv_downloads2onefolder()
monitor.make_datalist()
monitor.phasenet_picker(pnetobj)

01-11 19:29 [INFO] [json] running to create json
100%|██████████| 480/480 [00:54<00:00,  8.78it/s]  
01-11 19:29 [INFO] [json] ['CM-AGCC', 'CM-AGCC', 'CM-EZNC', 'CM-SNPBC', 'CM-OCNC', 'CM-SML1C', 'CM-VMM10', 'CM-VMM09', 'CM-VMM05', 'CM-VMM12', 'CM-VMM11', 'CM-VMM07', 'CM-BRR', 'CM-LL8C', 'CM-LL6C', 'CM-LL5C', 'CM-LL1C', 'CM-LL7C'] ok
01-11 19:29 [INFO] [json] Total time of execution: 2.642949 seconds
01-11 19:29 [INFO] [Downloader] Running to download
01-11 19:29 [INFO] [Downloader: chunktime] Total chunktime list: 1
01-11 19:29 [INFO] [Downloader: chunktime] 1 of 1 -> starttime=2017-10-15T06:00:00.000000Z | endtime=2017-10-15T08:00:00.000000Z
01-11 19:29 [INFO] [CM-VMM10: ] available:[('00', 'HH?')], selected [('00', 'HH?')] according to preference
01-11 19:29 [INFO] [CM-VMM12: ] available:[('00', 'HH?')], selected [('00', 'HH?')] according to preference
01-11 19:29 [INFO] [CM-AGCC: ] available:[('00', 'HH?')], selected [('00', 'HH?')] according to preference
01-11 19:29 [INFO] [CM-VM

### outs

In [6]:
pnet_detections = os.path.join(out_folder,"detections","pnet")
os.listdir(pnet_detections)

['picks_df_ok.csv', 'picks.csv', 'picks_df.csv', 'config.log']

In [9]:
picks = os.path.join(pnet_detections,'picks_df_ok.csv')
import pandas as pd
pd.read_csv(picks, nrows=10)

,Unnamed: 0,resource_id,time,uncertainty,lower_uncertainty,upper_uncertainty,confidence_level,network_code,station_code,channel_code,...,author_uri,creation_time,version,comments_text,mseed_starttime,mseed_endtime,sampling_rate,segment,sample,obs
0,0,20171015.060057.61-AIC-CM.VMM05.00.HHE,2017-10-15 06:00:57.610000+00:00,0.396209,NaN,NaN,39.620882,CM,VMM05,HHE,...,NaN,2022-01-12T01:23:13.982672Z,NaN,['20171015T060000Zto20171015T080000Z sr100.0 s...,2017-10-15 06:00:00+00:00,2017-10-15 08:00:00+00:00,100,3000,2761,single
1,1,20171015.060120.91-S-L2-CM.LL5C.00.HHE,2017-10-15 06:01:20.910000+00:00,0.395128,NaN,NaN,39.512850,CM,LL5C,HHE,...,NaN,2022-01-12T01:23:13.984786Z,NaN,['20171015T060000Zto20171015T080000Z sr100.0 s...,2017-10-15 06:00:00+00:00,2017-10-15 08:00:00+00:00,100,6000,2091,single
2,2,20171015.060129.20-AIC-CM.EZNC.00.HHE,2017-10-15 06:01:29.200000+00:00,0.308013,NaN,NaN,30.801252,CM,EZNC,HHE,...,NaN,2022-01-12T01:23:14.567461Z,NaN,['20171015T060000Zto20171015T080000Z sr100.0 s...,2017-10-15 06:00:00+00:00,2017-10-15 08:00:00+00:00,100,6000,2920,single
3,3,20171015.060130.85-S-L2-CM.LL5C.00.HHE,2017-10-15 06:01:30.850000+00:00,0.453498,NaN,NaN,45.349786,CM,LL5C,HHE,...,NaN,2022-01-12T01:23:13.986005Z,NaN,['20171015T060000Zto20171015T080000Z sr100.0 s...,2017-10-15 06:00:00+00:00,2017-10-15 08:00:00+00:00,100,9000,85,single
4,4,20171015.060159.21-AIC-CM.VMM09.00.HHE,2017-10-15 06:01:59.210000+00:00,0.422485,NaN,NaN,42.248460,CM,VMM09,HHE,...,NaN,2022-01-12T01:23:14.567099Z,NaN,['20171015T060000Zto20171015T080000Z sr100.0 s...,2017-10-15 06:00:00+00:00,2017-10-15 08:00:00+00:00,100,9000,2921,single
5,5,20171015.060228.23-AIC-CM.EZNC.00.HHE,2017-10-15 06:02:28.230000+00:00,0.434744,NaN,NaN,43.474448,CM,EZNC,HHE,...,NaN,2022-01-12T01:23:14.568706Z,NaN,['20171015T060000Zto20171015T080000Z sr100.0 s...,2017-10-15 06:00:00+00:00,2017-10-15 08:00:00+00:00,100,12000,2823,single
6,6,20171015.060504.62-S-L2-CM.LL5C.00.HHE,2017-10-15 06:05:04.620000+00:00,0.569916,NaN,NaN,56.991640,CM,LL5C,HHE,...,NaN,2022-01-12T01:23:13.993014Z,NaN,['20171015T060000Zto20171015T080000Z sr100.0 s...,2017-10-15 06:00:00+00:00,2017-10-15 08:00:00+00:00,100,30000,462,single
7,7,20171015.060513.92-S-L2-CM.LL5C.00.HHE,2017-10-15 06:05:13.920000+00:00,0.566873,NaN,NaN,56.687260,CM,LL5C,HHE,...,NaN,2022-01-12T01:23:13.993282Z,NaN,['20171015T060000Zto20171015T080000Z sr100.0 s...,2017-10-15 06:00:00+00:00,2017-10-15 08:00:00+00:00,100,30000,1392,single
8,8,20171015.060558.56-AIC-CM.VMM10.00.HHE,2017-10-15 06:05:58.560000+00:00,0.457307,NaN,NaN,45.730653,CM,VMM10,HHE,...,NaN,2022-01-12T01:23:13.994258Z,NaN,['20171015T060000Zto20171015T080000Z sr100.0 s...,2017-10-15 06:00:00+00:00,2017-10-15 08:00:00+00:00,100,33000,2856,single
9,9,20171015.060853.85-AIC-CM.VMM11.00.HHE,2017-10-15 06:08:53.850000+00:00,0.523188,NaN,NaN,52.318790,CM,VMM11,HHE,...,NaN,2022-01-12T01:23:14.575228Z,NaN,['20171015T060000Zto20171015T080000Z sr100.0 s...,2017-10-15 06:00:00+00:00,2017-10-15 08:00:00+00:00,100,51000,2385,single
